In [15]:
import numpy as np
from IPython.display import display, Markdown
from meowmeow import (
    Mat, Vec, display_latex, _err,
    _as_array_M, _as_array_v,
    gauss_solve, gauss_ops_theory,
    _ops_zero, _ops_inc, _ops_merge,
)

In [36]:
def tridiag(a, b, c):
    a = np.asarray(a, dtype=float).reshape(-1)
    b = np.asarray(b, dtype=float).reshape(-1)
    c = np.asarray(c, dtype=float).reshape(-1)
    n = b.size
    if a.size != n or c.size != n:
        raise _err("ожидались три вектора одинаковой длины для диагоналей (a,b,c)")
    A = np.zeros((n, n), dtype=float)
    i = np.arange(n)
    A[i, i] = b
    A[i[1:], i[:-1]] = a[1:]
    A[i[:-1], i[1:]] = c[:-1]
    return Mat(A)

In [ ]:
def check_theorem_conditions(a, b, c):
    a = np.asarray(a, float).reshape(-1)
    b = np.asarray(b, float).reshape(-1)
    c = np.asarray(c, float).reshape(-1)
    n = b.size
    ok1 = np.all(np.abs(b) >= (np.abs(a) + np.abs(c)))
    ok2 = np.all(np.abs(b) > np.abs(a))
    ok3 = np.isclose(a[0], 0.0)
    ok4 = np.isclose(c[-1], 0.0)
    return {
        "diag_dom": bool(ok1),
        "b_gt_a": bool(ok2),
        "a1_is_0": bool(ok3),
        "cn_is_0": bool(ok4),
        "all_ok": bool(ok1 and ok2 and ok3 and ok4),
    }

In [28]:
def _print_conditions_report(a, b, c):
    chk = check_theorem_conditions(a,b,c)
    bullet = lambda ok: "☑" if ok else "☒"
    display(Markdown("**проверка достаточных условий теоремы:**"))
    display(Markdown(f"{bullet(chk['diag_dom'])}: " 
                     r"$|b_k|\ge|a_k|+|c_k|$"))
    display(Markdown(f"{bullet(chk['b_gt_a'])}: " 
                     r"$|b_k|>|a_k|$"))
    display(Markdown(f"{bullet(chk['a1_is_0'])} $a_1=0$"))
    display(Markdown(f"{bullet(chk['cn_is_0'])} $c_n=0$"))
    if chk["all_ok"]:
        display(Markdown("**итог:** условия выполнены; прямая прогонка пройдёт до конца (=^..^=)"))
    else:
        display(Markdown("**итог:** *не все* условия выполнены; прогонка может упасть (=^._.^=)"))

In [19]:
def thomas_solve(a_in, b_in, c_in, d_in):
    a = np.asarray(a_in, float).reshape(-1)
    b = np.asarray(b_in, float).reshape(-1)
    c = np.asarray(c_in, float).reshape(-1)
    d = _as_array_v(d_in)
    n = b.size
    if not (a.size == b.size == c.size == d.size):
        raise _err("ожидались четыре вектора одинаковой длины (a,b,c,d)")
    if n < 2:
        raise _err("n должно быть ≥ 2")

    ops = _ops_zero()

    alpha = np.zeros(n, dtype=float)
    beta = np.zeros(n, dtype=float)
    gamma = np.zeros(n, dtype=float)
    x = np.zeros(n, dtype=float)

    gamma[0] = b[0]
    if gamma[0] == 0.0:
        raise _err("γ1 = b1 = 0; прогонка невозможна")
    alpha[0] = -c[0]/gamma[0]; _ops_inc(ops,"div")
    beta[0] = d[0]/gamma[0]; _ops_inc(ops,"div")

    for i in range(1, n-1):
        gamma[i] = b[i] + a[i]*alpha[i-1]; _ops_inc(ops,"mul"); _ops_inc(ops,"add")
        if gamma[i] == 0.0:
            raise _err(f"γ_{i+1}=0; прогонка останавливается на шаге i={i+1}")
        alpha[i] = - c[i] / gamma[i];      _ops_inc(ops,"div")
        beta[i]  = (d[i] - a[i]*beta[i-1]) / gamma[i]; _ops_inc(ops,"mul"); _ops_inc(ops,"sub"); _ops_inc(ops,"div")

    gamma[n-1] = b[n-1] + a[n-1]*alpha[n-2]; _ops_inc(ops,"mul"); _ops_inc(ops,"add")
    if gamma[n-1] == 0.0:
        raise _err("γ_n = 0; последний шаг прямого хода невозможен")
    beta[n-1] = (d[n-1] - a[n-1]*beta[n-2]) / gamma[n-1]; _ops_inc(ops,"mul"); _ops_inc(ops,"sub"); _ops_inc(ops,"div")

    x[n-1] = beta[n-1]
    for i in range(n-2, -1, -1):
        x[i] = alpha[i]*x[i+1] + beta[i]; _ops_inc(ops,"mul"); _ops_inc(ops,"add")

    ops["total"] = sum(ops[k] for k in ops if k != "total")

    info = {
        "alpha": alpha, "beta": beta, "gamma": gamma,
        "ops": ops,
        "max_abs_alpha": float(np.max(np.abs(alpha))),
        "ok_abs_alpha_le_1": bool(np.max(np.abs(alpha)) <= 1.0 + 1e-12),
    }
    return Vec(x), info

In [37]:
def make_tridiag_dominant(n: int, a_scale=0.3, c_scale=0.3, seed: int|None = 42, with_solution=True, x_range=(-10.0, 10.0)):
    rng = np.random.default_rng(seed)
    a = rng.uniform(-a_scale, a_scale, size=n)
    c = rng.uniform(-c_scale, c_scale, size=n)
    a[0] = 0.0
    c[-1] = 0.0
    b = (np.abs(a) + np.abs(c)) + rng.uniform(1.0, 2.0, size=n)

    A = tridiag(a, b, c)
    if with_solution:
        lo, hi = x_range
        x_true = rng.uniform(lo, hi, size=n)
        d = A.data @ x_true
        return (a, b, c, Vec(d), Vec(x_true), A)
    else:
        d = rng.uniform(-5.0, 5.0, size=n)
        return (a, b, c, Vec(d), None, A)

In [22]:
def make_poisson1d_system(n: int, f_kind: str = "zero", a0: float = 0.0, b1: float = 0.0):
    h = 1.0/n
    x = np.linspace(0.0, 1.0, n+1)
    a = np.zeros(n+1); b = np.zeros(n+1); c = np.zeros(n+1); d = np.zeros(n+1)
    b[0] = 1.0; d[0] = a0
    b[-1] = 1.0; d[-1] = b1
    for k in range(1, n):
        a[k] = -1.0/h**2
        b[k] = 1.0 + 2.0/h**2
        c[k] = -1.0/h**2
        if f_kind == "zero":
            d[k] = 0.0
        elif f_kind == "sinpi":
            d[k] = np.sin(np.pi * x[k])
        elif f_kind == "ones":
            d[k] = 1.0
        else:
            d[k] = 0.0
    A = tridiag(a, b, c)
    return (a, b, c, Vec(d), A, x)

In [30]:
def _metrics_after(A: Mat, x_hat: Vec, d: Vec, x_true: Vec|None = None):
    A_arr = _as_array_M(A); xh = _as_array_v(x_hat); d_arr = _as_array_v(d)
    r = A_arr @ xh - d_arr
    num = float(np.linalg.norm(r, 2))
    normA = float(np.linalg.norm(A_arr, 2))
    normx = float(np.linalg.norm(xh, 2))
    rel_res = num / (normA * normx) if normA*normx != 0 else float("inf")
    display(Markdown(
        fr"$\mathrm{{rel\_res}}=\dfrac{{\lVert A x-d\rVert_2}}{{\lVert A\rVert_2\,\lVert x\rVert_2}}"
        fr"=\dfrac{{{num:.6g}}}{{{normA:.6g}\cdot{normx:.6g}}}"
        fr"={rel_res:.6g}$"
    ))
    display_latex(num, label=r"\|r\|_2")
    if x_true is not None:
        xt = _as_array_v(x_true)
        err = xh - xt
        num = float(np.linalg.norm(err, 2))
        den = float(np.linalg.norm(xt, 2))
        rel_err = num / den
        display(Markdown(
            fr"$\mathrm{{rel\_err}}=\frac{{\|\hat x-x\|_2}}{{\|x\|_2}}"
            fr"=\frac{{{num:.6g}}}{{{den:.6g}}}"
            fr"={rel_err:.6g}$"))

In [23]:
def compare_with_gauss(A: Mat, d: Vec, xhat_thomas: Vec, ops_thomas: dict, pivot="col"):
    display(Markdown("**сравнение с Гауссом**"))
    xg, info = gauss_solve(A, d, pivot=pivot)
    display_latex(xg, label=r"\hat x_{\mathrm{gauss}}")
    A_arr = _as_array_M(A); dg = _as_array_v(d)
    rg = A_arr @ xg.data - dg
    num = float(np.linalg.norm(rg, 2))
    normA = float(np.linalg.norm(A_arr, 2))
    normx = float(np.linalg.norm(xg.data, 2))
    rel_res_g = num / (normA * normx) if normA*normx != 0 else float("inf")
    display(Markdown(
        fr"$\mathrm{{rel\_res}}^{{(\mathrm{{gauss}})}}=\dfrac{{\lVert A\,\hat x-b\rVert_2}}{{\lVert A\rVert_2\,\lVert \hat x\rVert_2}}"
        fr"=\dfrac{{{num:.6g}}}{{{normA:.6g}\cdot{normx:.6g}}}"
        fr"={rel_res_g:.6g}$"
    ))

    display(Markdown("**операции:**"))
    display(Markdown(f"Томас: `Q = {ops_thomas['total']}`"))
    display(Markdown(f"Гаусс: `Q = {info['ops']['total']}`"))

In [24]:
def _read_tridiag_by_hand():
    n = int(input("введите n (размерность): ").strip())
    print("введите вектор a (нижняя диагональ): n чисел через пробел (ожидается a1=0)")
    a = np.array([float(x) for x in input("a: ").replace(",", " ").split()], dtype=float)
    print("введите вектор b (главная диагональ): n чисел через пробел")
    b = np.array([float(x) for x in input("b: ").replace(",", " ").split()], dtype=float)
    print("введите вектор c (верхняя диагональ): n чисел через пробел (ожидается c_n=0)")
    c = np.array([float(x) for x in input("c: ").replace(",", " ").split()], dtype=float)
    print("введите правую часть d: n чисел через пробел")
    d = np.array([float(x) for x in input("d: ").replace(",", " ").split()], dtype=float)
    if not (len(a)==len(b)==len(c)==len(d)==n):
        raise _err("длины (a,b,c,d) должны совпадать с n")
    return a,b,c,Vec(d),tridiag(a,b,c)

In [7]:
def mini_demo(n=6):
    display(Markdown("**1. уютная система**"))
    a,b,c,d,x_true,A = make_tridiag_dominant(n, seed=7)
    display_latex(A, label=r"A^{(1)}")
    _print_conditions_report(a,b,c)
    x1, info1 = thomas_solve(a,b,c,d)
    display_latex(x1, label=r"x^{(1)}_{\mathrm{Thomas}}")
    _metrics_after(A, x1, d, x_true=x_true)
    display(Markdown(f"max $|\\alpha|$ = `{info1['max_abs_alpha']:.6g}`; "
                     f"устойчивость обратного хода: `{info1['ok_abs_alpha_le_1']}`"))
    display(Markdown(f"`Q_meas (Thomas)` = **{info1['ops']['total']}**"))

    display(Markdown("**2. нарушены условия (18)**"))
    rng = np.random.default_rng(123)
    a2 = rng.uniform(-1,1,size=n); c2 = rng.uniform(-1,1,size=n); a2[0]=0.0; c2[-1]=0.0
    b2 = rng.uniform(0.1, 0.5, size=n)  # специально слишком маленькая диагональ (=^..^=)
    A2 = tridiag(a2,b2,c2)
    d2 = Vec(rng.uniform(-2,2,size=n))
    display_latex(A2, label=r"A^{(2)}")
    _print_conditions_report(a2,b2,c2)
    try:
        x2, info2 = thomas_solve(a2,b2,c2,d2)
        display_latex(x2, label=r"x^{(2)}_{\mathrm{Thomas}}")
        _metrics_after(A2, x2, d2, x_true=None)
    except Exception as e:
        display(Markdown(f"ожидаемо упало: `{e}`"))

    display(Markdown("**3. из примера (6) на стр.2: $-u''+u=f$, $u(0)=a$, $u(1)=b$**"))
    a3,b3,c3,d3,A3,_ = make_poisson1d_system(n, f_kind="sinpi", a0=0.0, b1=0.0)
    display_latex(A3, label=r"A^{(3)}")
    _print_conditions_report(a3,b3,c3)
    x3, info3 = thomas_solve(a3,b3,c3,d3)
    display_latex(x3, label=r"x^{(3)}_{\mathrm{Thomas}}")
    _metrics_after(A3, x3, d3, x_true=None)
    display(Markdown(f"`Q_meas (Thomas)` = **{info3['ops']['total']}**"))
    print("мяу, готово (=^..^=)")

In [26]:
def main():
    display(Markdown(
        "мяу мяу, что делаем?\n\n"
        "[1] решить $A x=d$ методом прогонки (ввод вручную / случайная генерация)\n\n"
        "[2] глазеем сразу на три случая (корректный / нарушены условия (18) / из примера (6) на стр.2)\n"
    ))
    mode = (input("ваш выбор (1/2): ").strip() or "2")

    if mode == "2":
        try:
            n = int(input("n (по умолчанию 6): ").strip() or "6")
        except Exception:
            n = 6
        mini_demo(n=n)
        return

    if mode != "1":
        print("мяу, нужно выбрать 1 или 2")
        return

    how = (input("как задаём систему? (m=вручную / d=случайная, для которой верны тождества (18) / p=из примера (6) на стр.2): ").strip().lower() or "m")

    x_true = None
    if how.startswith("m"):
        a,b,c,d,A = _read_tridiag_by_hand()
    elif how.startswith("d"):
        n = int(input("n = ").strip())
        a,b,c,d,x_true,A = make_tridiag_dominant(n, seed=42, with_solution=True)
        display_latex(A, label=r"A")
        display_latex(x_true, label=r"x_{\mathrm{true}}")
    else:
        n = int(input("n (число шагов по интервалу, итоговый размер = n+1) = ").strip())
        fk = (input("f(x): zero/sinpi/ones [по умолчанию zero]: ").strip().lower() or "zero")
        a0 = float((input("u(0)=a [0]: ").strip() or "0"))
        b1 = float((input("u(1)=b [0]: ").strip() or "0"))
        a,b,c,d,A,_ = make_poisson1d_system(n, f_kind=fk, a0=a0, b1=b1)
        display_latex(A, label=r"A")

    _print_conditions_report(a,b,c)

    display(Markdown("**прогонка**"))
    x_hat, info = thomas_solve(a,b,c,d)
    display_latex(Vec(a), label=r"a")
    display_latex(Vec(b), label=r"b")
    display_latex(Vec(c), label=r"c")
    display_latex(d, label=r"d")
    display_latex(tridiag(a,b,c), label=r"A")
    display_latex(x_hat, label=r"\hat x_{\mathrm{Thomas}}")

    display(Markdown(f"max $|\\alpha|$ = `{info['max_abs_alpha']:.6g}`; "
                     f"устойчивость обратного хода: `{info['ok_abs_alpha_le_1']}`"))

    _metrics_after(tridiag(a,b,c), x_hat, d, x_true=x_true)

    display(Markdown("**операции**"))
    display(Markdown(f"`Q_meas (Thomas)` = **{info['ops']['total']}**"))
    q_thomas_theory = 8*len(b)
    display(Markdown(f"`Q_theory (Thomas)` ≈ `8n = {q_thomas_theory}`"))

    want_cmp = (input("сравнить с Гауссом по операциям? (y/n, по умолчанию y): ")
                .strip().lower() or "y").startswith("y")
    if want_cmp:
        compare_with_gauss(tridiag(a,b,c), d, x_hat, info["ops"], pivot="col")

    print("мяу, готово (=^..^=)")

In [35]:
main()

мяу мяу, что делаем?

[1] решить $A x=d$ методом прогонки (ввод вручную / случайная генерация)

[2] глазеем сразу на три случая (корректный / нарушены условия (18) / из примера (6) на стр.2)


**1. уютная система**

<IPython.core.display.Math object>

**проверка достаточных условий теоремы:**

☑: $|b_k|\ge|a_k|+|c_k|$

...

<IPython.core.display.Math object>

$\mathrm{rel\_res}=\dfrac{\lVert A x-d\rVert_2}{\lVert A\rVert_2\,\lVert x\rVert_2}=\dfrac{9.56662e-16}{136.216\cdot0.162692}=4.31682e-17$

<IPython.core.display.Math object>

`Q_meas (Thomas)` = **49**

мяу, готово (=^..^=)
